In [20]:
template = """
## {subject_of_edit} and {related_entity}
**Subject of Edit:** {subject_of_edit}

**Edit made:** {edit_made} {target_new} (previously {target_old})

### **Ground Truth about subject of edit ({subject_of_edit}):**
{ground_truth_about_subject_of_edit}

### **Ground Truth about related entity ({related_entity}):**
{ground_truth_about_related_entity}

### **Passage of text about subject of edit ({subject_of_edit}):**
{passage_of_text_about_subject_of_edit}

### **Passage of text about related entity ({related_entity}):** 
{passage_of_text_about_related_entity}
"""

import json
import os

BASE_DIR = '../data/generated_samples_all'
GREEDY_DIR = os.path.join(BASE_DIR, 'model_llama2-chat_no_edit_False_use_sampling_False_token_length_1024_method_rome')
SAMPLED_DIR = os.path.join(BASE_DIR, 'model_llama2-chat_no_edit_False_use_sampling_True_token_length_1024_method_rome')
GREEDY_NO_EDIT_DIR = os.path.join(BASE_DIR, 'model_llama2-chat_no_edit_True_use_sampling_False_token_length_1024_method_rome')
SAMPLED_NO_EDIT_DIR = os.path.join(BASE_DIR, 'model_llama2-chat_no_edit_True_use_sampling_True_token_length_1024_method_rome')

def get_samples_from_dir(dir_path):
    samples = []
    for file_name in os.listdir(dir_path):
        with open(os.path.join(dir_path, file_name), 'r') as f:
            samples.append(json.load(f))
    return samples

greedy_samples = get_samples_from_dir(GREEDY_DIR)
sampled_samples = get_samples_from_dir(SAMPLED_DIR)
greedy_no_edit_samples = get_samples_from_dir(GREEDY_NO_EDIT_DIR)
sampled_no_edit_samples = get_samples_from_dir(SAMPLED_NO_EDIT_DIR)

def get_sample_text(sample):
    subject_ground_truth = sample['dependancies']['subject_entity']['ground_truth']
    subject_ground_truth_string = '- ' + '\n- '.join([f"{key}: {', '.join(value)}" for key,value in subject_ground_truth.items()])
    related_entity_ground_truth = sample['dependancies']['coupled_entities'][0]['ground_truth']
    related_entity_ground_truth_string = '- ' + '\n- '.join([f"{key}: {', '.join(value)}" for key,value in related_entity_ground_truth.items()])
    return template.format(
        subject_of_edit=sample["requested_rewrite"]['subject'],
        edit_made=sample["requested_rewrite"]["prompt"].format(
            sample["requested_rewrite"]['subject']
        ),
        ground_truth_about_subject_of_edit=subject_ground_truth_string.strip(),
        ground_truth_about_related_entity=related_entity_ground_truth_string.strip(),
        passage_of_text_about_subject_of_edit=sample['subject_prompt_with_relationship'].strip(),
        passage_of_text_about_related_entity=sample['coupled_prompt_with_relationship'].strip(),
        related_entity=sample['dependancies']['coupled_entities'][0]['entity'],
        target_new=sample["requested_rewrite"]['target_new']['str'],
        target_old=sample["requested_rewrite"]['target_true']['str']
    )

def get_samples_text(samples):
    return '\n\n'.join([get_sample_text(sample) for sample in samples])

greedy_samples_text = get_samples_text(greedy_samples)
sampled_samples_text = get_samples_text(sampled_samples)
greedy_no_edit_samples_text = get_samples_text(greedy_no_edit_samples)
sampled_no_edit_samples_text = get_samples_text(sampled_no_edit_samples)

with open(os.path.join('../data', 'evaluation_samples', 'greedy_samples.md'), 'w') as f:
    f.write(greedy_samples_text)

with open(os.path.join('../data', 'evaluation_samples', 'sampled_samples.md'), 'w') as f:
    f.write(sampled_samples_text)

with open(os.path.join('../data', 'evaluation_samples', 'greedy_no_edit_samples.md'), 'w') as f:
    f.write(greedy_no_edit_samples_text)

with open(os.path.join('../data', 'evaluation_samples', 'sampled_no_edit_samples.md'), 'w') as f:
    f.write(sampled_no_edit_samples_text)